In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn import metrics
import joblib
import datetime
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from model.NN import NN, DatasetUtil

n_days_lookahead = int(input('Please input the length of days lookahead in {5, 7, 15, 30, 45, 60, 90, 120}: '))

if(n_days_lookahead not in [5, 7, 15, 30, 45, 60, 90, 120]):
    print('Input does not meet requirements.')
    exit()

data_type = str(input('Please specify the coverage of the data {A - Manufacturer 1, B - Manufacturer 2, C - Manufacturer 1 & 2}: '))

if(data_type not in ['A', 'B', 'C']):
    print('Input does not meet requirements.')
    exit()

dit_str = {'A': 'mc1', 'B': 'mc2', 'C': 'mc1_mc2'}

model_type = str(input('Please input the type of trained model to use {A - Manufacturer 1, B - Manufacturer 2, C - Manufacturer 1 & 2}: '))

if(model_type not in ['A', 'B', 'C']):
    print('Input does not meet requirements.')
    exit()


def loadData():

    X = np.load('../data/' + dit_str[data_type] + '/' + str(n_days_lookahead) + '_days_lookahead/smart_test.npy',allow_pickle=True)
    y = np.load('../data/' + dit_str[data_type] + '/' + str(n_days_lookahead) + '_days_lookahead/test_labels.npy',allow_pickle=True)

    X = X.astype('float32')
    y = y.astype('float32')
    return X, y


def get_all_metrics(true, predicted, score):
    confusion_matrix = metrics.confusion_matrix(true, predicted)
    fpr_list, tpr_list, thersholds = roc_curve(true, score)
    roc_auc = auc(fpr_list, tpr_list)
    TP = confusion_matrix[0][0]
    FN = confusion_matrix[0][1]
    FP = confusion_matrix[1][0]
    TN =  confusion_matrix[1][1]
    precision_of_failed = TP / (TP + FP)
    precision_of_healthy = TN / (TN + FN)
    tpr = TP / (TP + FN)
    fpr = FP / (TN + FP)
    auc_score = roc_auc
    f1_score = 2*precision_of_failed*tpr / (precision_of_failed+tpr)
    print('precision of failed: ', precision_of_failed)
    print('precision of healthy: ', precision_of_healthy)
    print('tpr: ', tpr)
    print('fpr: ', fpr)
    print('auc: ', auc_score)
    print('f1-score: ', f1_score)
    print('roc curve: ')
    plt.plot(fpr_list, tpr_list)
    plt.xlim([-0.05, 1.05])  
    plt.ylim([-0.05, 1.05])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
    
# print('-------------------- NN --------------------')
# print('------------------ Loading Data ------------------')
X, y = loadData()

input_size = 330
hidden_size = 512

output_size = 1
num_layers = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = NN(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)
model.load_state_dict(torch.load('../trained_model/'+ dit_str[model_type] + '/' + str(n_days_lookahead) + '_days_lookahead/nn_online.pth'))

X = X.reshape((len(X),-1))

test_dataset = DatasetUtil(X, y)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

y_pred_list = []
y_true_list = []
y_score_list = []
with torch.no_grad():
    total=0
    for i, (X, y) in enumerate(test_loader):
        X = X.to(device)
        y = y.to(device)
        score = model(X)
        predicted = torch.tensor([1 if x[0] > 0.5 else 0 for x in score]).to(device)
        for j in range(0,len(y)):
            y_pred_list.append(int(predicted[j].cpu()))
            y_true_list.append(y[j].cpu())
            y_score_list.append(float(score[j][0]))

get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray(y_score_list))


/tmp/ipykernel_75318/1706291421.py:118: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray(y_score_list))
/tmp/ipykernel_75318/1706291421.py:118: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray

precision of failed:  0.8403604216252976
precision of healthy:  0.8272311212814645
tpr:  0.8238333333333333
fpr:  0.1565
auc:  0.9066036527777778
f1-score:  0.8320148123211581


/tmp/ipykernel_75318/1706291421.py:118: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray(y_score_list))
/tmp/ipykernel_75318/1706291421.py:118: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray

precision of failed:  0.7296476306196841
precision of healthy:  0.7791728212703102
tpr:  0.8006666666666666
fpr:  0.2966666666666667
auc:  0.8252216388888889
f1-score:  0.7635092180546724


/tmp/ipykernel_75318/1706291421.py:118: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray(y_score_list))
/tmp/ipykernel_75318/1706291421.py:118: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray

...

/tmp/ipykernel_75318/1706291421.py:118: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray(y_score_list))
/tmp/ipykernel_75318/1706291421.py:118: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray

precision of failed:  0.7516660820764643
precision of healthy:  0.7278501111463956
tpr:  0.7143333333333334
fpr:  0.236
auc:  0.8118528611111111
f1-score:  0.7325243548111434


/tmp/ipykernel_75318/1706291421.py:118: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray(y_score_list))
/tmp/ipykernel_75318/1706291421.py:118: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray

precision of failed:  0.7154992548435172
precision of healthy:  0.7733459357277883
tpr:  0.8001666666666667
fpr:  0.31816666666666665
auc:  0.8097740833333333
f1-score:  0.7554681353265146
precision of failed:  0.7122930717351318
precision of healthy:  0.7529218407596786
tpr:  0.7745
fpr:  0.31283333333333335
auc:  0.8013905138888889
f1-score:  0.7420951772596615


/tmp/ipykernel_75318/1706291421.py:118: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray(y_score_list))
/tmp/ipykernel_75318/1706291421.py:118: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  get_all_metrics(np.asarray(y_true_list).astype('int'), np.asarray(y_pred_list), np.asarray